In [51]:
import pandas as pd

cnn_path = 'ensemble_CNN20_res.csv'
spy_path = 'sp500_filtered_stock.csv'
ticker_path = 'sp500_filtered_stock_matched_tickers.csv'

spy = pd.read_csv(spy_path)
cnn = pd.read_csv(cnn_path)
ticker = pd.read_csv(ticker_path)

# RET를 값으로, Date를 인덱스로, PERMNO를 컬럼으로 하는 데이터프레임 생성
spy_ret = spy.pivot(index='date', columns='PERMNO', values='RET')
spy_ret

# NaN값을 -2으로 채움
spy_ret = spy_ret.fillna(0)
spy_ret

universe = spy_ret.columns

# TICKER와 StockID를 이용하여 spy_ret의 컬럼명을 일치하는 TICKER로 매핑

ticker

# dict로 변환
ticker_dict = dict(zip(ticker['StockID'], ticker['TICKER']))
ticker_dict

# spy_ret의 컬럼명을 ticker_dict로 매핑
spy_ret = spy_ret.rename(columns=ticker_dict)
spy_ret.to_csv('return_df.csv')

# investment_date 를 인덱스로, up_prob를 값으로, StockID를 컬럼으로 하는 데이터프레임을 만들어주세요.
cnn_pivot = cnn.pivot(index='investment_date', columns='StockID', values='up_prob')

# StockID가 universe에 있는 것만 남김
cnn_pivot = cnn_pivot[universe]
cnn_pivot

# NaN 값은 그 행의 평균값으로 채움
# NaN 값이 있더라도 행으로 평균을 내서 채우기
cnn_pivot2 = cnn_pivot.apply(lambda x: x.fillna(x.mean()), axis=1)
cnn_pivot2 = cnn_pivot2.rename(columns=ticker_dict)
cnn_pivot2

from sklearn.preprocessing import MinMaxScaler

# 행 단위 Min-Max 정규화 함수 정의
def row_min_max_normalize(df: pd.DataFrame) -> pd.DataFrame:
    # 정규화된 데이터를 저장할 DataFrame
    normalized_df = pd.DataFrame(index=df.index, columns=df.columns)
    
    # 각 행을 개별적으로 정규화
    for idx in df.index:
        row = df.loc[idx].values.reshape(-1, 1)  # 행 데이터를 2D로 변환
        scaler = MinMaxScaler(feature_range=(0, 1))
        normalized_row = scaler.fit_transform(row).flatten()  # 행 단위 정규화
        normalized_df.loc[idx] = normalized_row

    return normalized_df

# 예시 실행
normalized_up_prob_data = row_min_max_normalize(cnn_pivot2)
normalized_up_prob_data.to_csv('up_prob_df.csv')